In [ ]:
import pandas as pd 

df = pd.read_csv('data/data_to_simulate.csv', index_col=0)
df.dropna(inplace=True)
print(df.shape)
columns = ['GTIME_KI', 'GSTATUS_KI', 'AGE_DON', 'HCV_SEROSTATUS', 'GENDER_DON', 'AGE', 'DIAB',
                'BMI_CALC', 'ETHCAT', 'GENDER', 'COLD_ISCH_KI', 'SERUM_CREAT',
                'CREAT_TRR', 'delta_age', 'HIST_DIABETES_DON', 'HGT_CM_DON_CALC',
                'COD_CAD_DON', 'WGT_KG_DON_CALC', 'CREAT_DON', 'ETHCAT_DON',
                'HIST_HYPERTENS_DON', 'NON_HRT_DON', 'DIALYSIS_DATE','TX_DATE', 'PT_CODE', 'ABO', 'ABO_DON']

predictors = ['AGE_DON', 'CREAT_TRR', 'HGT_CM_DON_CALC', 'HCV_SEROSTATUS', 'WGT_KG_DON_CALC',
                    'HIST_DIABETES_DON', 'HIST_HYPERTENS_DON', 'COD_CAD_DON', 'ETHCAT_DON', 'ABO_MAT', 
                    'HLAMIS', 'DIALYSIS_DATE', 'AGE', 'DIAB']

donor = df[['AGE_DON', 'CREAT_TRR', 'HGT_CM_DON_CALC', 'HCV_SEROSTATUS', 'HIST_DIABETES_DON', 'HIST_HYPERTENS_DON', 'ABO_DON',
             'AGE_DONge50', 'CREAT_TRRge1.5', 'SERUM_CREATg1.2', 'WGT_KG_DON_CALCl80', 'COD_CAD_DON_2', 'ETHCAT_DON_2']]

donor.to_csv('data/donors.csv')

recipient = df[[ 'DIALYSIS_DATE', 'AGE', 'DIAB', 'ETHCAT', 'GENDER', 'ABO']]
recipient.to_csv('data/recipients.csv')

(22250, 41)


In [2]:
recipient.ABO.unique()

array(['B', 'O', 'A', 'A1', 'AB', 'A1B', 'A2', 'A2B'], dtype=object)

In [3]:
blood_type_compatibility = {
    "A": ["A", "A1", "A2", "AB", "A1B", "A2B"],
    "A1": ["A", "A1", "A2", "AB", "A1B", "A2B"],
    "A2": ["A", "A1", "A2", "AB", "A1B", "A2B", "B"],
    "B":  ["B", "AB", "A1B", "A2B"],
    "AB": ["AB", "A1B", "A2B"],
    "A1B": ["A1B"],
    "A2B": ["A2B", "B"],
    "O": ["A", "A1", "A2", "B", "AB", "A1B", "A2B", "O"]
}

In [4]:
from pathlib import Path
import os
import matplotlib.pyplot as plt
path = Path("C:/Users/HP/Desktop/UNOS data/codes/simulation-waitline-unos").resolve()  # Get the absolute path
os.chdir(path)
import pandas as pd
import numpy as np
import scipy as sc
import recipient as rec
import prediction as pred 
import donor as don

MAX_WAIT_TIME = 5 * 365
TARGET_TIME  = 5 * 365
SIMULATION_TIME = 10* 365 
predictor = pred.SurvivalPredictionModel(model_path='models\decision_tree.pkl')
#9.1% leave the list
theshold_leave_list = 0.091 
recipients_for_initialization = 100
time_before_starting_simulation = 0
released_recipients = []

df = pd.DataFrame(columns=['replicate', 'ethcat', 'gender', 'waiting_time'])

def run_simulation(T=10, replicates=1, verbose=False):
    
    average_waiting_times = []
    number_of_matches_list = []
    
    for r in range(replicates):
        
        print('Replicate #', str(r))
        wait_list = []  # Initialize an empty waitlist for each replicate
        leave_list = []
        total_waiting_time = 0  # Track the total waiting time for this replicate
        num_matched_patients = 0  # Track the number of matched patients
        current_t = 0  # Start time    
    
        for i in range(recipients_for_initialization):
                new_recipient = rec.Recipient(time_before_starting_simulation) 
                new_recipient.create_recipient()
                wait_list.append(new_recipient)
        
        donor_arrival_times = list(np.random.exponential(scale=10, size=100000))
        recipient_arrival_times = list(np.random.exponential(scale=5, size=100000))
        
        while current_t < T:
                       
            if donor_arrival_times[0] < recipient_arrival_times[0]:
                current_t += donor_arrival_times.pop(0)
                donor = don.Donor(current_t)
                donor.create_donor()
                if verbose: 
                    donor.show()
                if len(wait_list) > 0:
                    patient_who_leaves_list = np.random.choice(len(wait_list))
                    leave_list_random = np.random.random()
                    if theshold_leave_list > leave_list_random:
                        out_of_list = wait_list.pop(patient_who_leaves_list)
                        leave_list.append(out_of_list)
                        print('One patient left the waitlist.')   
                                                 
                    survival_probs = [predictor.predict_survival_prob(recipient, donor, TARGET_TIME) 
                                      for recipient in wait_list 
                                      if donor.blood in blood_type_compatibility[recipient.blood]]         
                               
                    best_match_index = np.argmax(survival_probs)
                    best_patient = wait_list.pop(best_match_index)
                    
                    print('A donation was done. Day: ', str(current_t))
                    waiting_time = current_t - best_patient.arrival_time
                    total_waiting_time += waiting_time
                    best_patient.waiting_time = waiting_time
                    
                    df.loc[len(df)] = [r, best_patient.ethcat, best_patient.gender, best_patient.waiting_time]
                    
                    released_recipients.append(best_patient)
                    num_matched_patients += 1
            else: 
                current_t += recipient_arrival_times.pop(0)  
                new_recipient = rec.Recipient(current_t) 
                new_recipient.create_recipient()
                
                if verbose:
                    new_recipient.show() 
                    
                wait_list.append(new_recipient)
            if num_matched_patients>0:
                average_time = total_waiting_time / num_matched_patients
            else: 
                average_time = 'INF'
            if verbose:
                print(f'Number of matches: {num_matched_patients}, recipients in wait list: {len(wait_list)}, average time in waitlist {average_time}')
        if num_matched_patients > 0:
            average_waiting_times.append(total_waiting_time / num_matched_patients)
            number_of_matches_list.append(num_matched_patients)
        else:
            average_waiting_times.append(0)
    return average_waiting_times, number_of_matches_list


<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
C:\Users\HP\AppData\Local\Temp\ipykernel_22964\946184350.py:16: SyntaxWarning: invalid escape sequence '\d'
  predictor = pred.SurvivalPredictionModel(model_path='models\decision_tree.pkl')


Model loaded successfully from models\decision_tree.pkl.


In [5]:
run_simulation(T=SIMULATION_TIME, replicates=30)

Replicate # 0
A donation was done. Day:  47.499013510676455
A donation was done. Day:  53.46134856619025
A donation was done. Day:  58.93488269645603
One patient left the waitlist.
A donation was done. Day:  71.51235607864078
A donation was done. Day:  74.34318963656584
A donation was done. Day:  83.65980932399307
One patient left the waitlist.
A donation was done. Day:  90.61962079612533
A donation was done. Day:  95.40127554187161
A donation was done. Day:  382.9430424481003
A donation was done. Day:  389.88416336731825
A donation was done. Day:  411.0199230443368
A donation was done. Day:  416.6042655928992
A donation was done. Day:  431.0606021497009
A donation was done. Day:  441.55079233760364
A donation was done. Day:  443.5401976312728
A donation was done. Day:  454.96247564713553
A donation was done. Day:  464.49768074446075
A donation was done. Day:  469.5688053751545
One patient left the waitlist.
A donation was done. Day:  474.54936096070475
A donation was done. Day:  482.0

([368.0946178119169,
  455.444060134461,
  54.480872961808124,
  404.39190374177366,
  886.2798354178906,
  322.19689698948844,
  165.96130176818443,
  590.9009771310339,
  1024.2053592078503,
  1372.7735336989958,
  161.05418212076577,
  74.86711868615433,
  688.4926042745939,
  345.8159802754249,
  538.0655159668862,
  418.15611910277687,
  132.32063789128762,
  747.2657460548073,
  2093.75495775612,
  1483.1705504238355,
  839.2420894111056,
  2551.861076924845,
  967.135430759779,
  467.20110498352767,
  14.456250810609612,
  48.09130526418697,
  721.4983881053474,
  1350.3543093232013,
  187.38810407421454,
  809.2232264805785],
 [25,
  28,
  7,
  14,
  56,
  57,
  15,
  20,
  74,
  165,
  26,
  14,
  24,
  12,
  19,
  51,
  26,
  27,
  72,
  35,
  71,
  47,
  104,
  13,
  3,
  14,
  14,
  52,
  21,
  47])

In [6]:
ethcat_mapping = {
    1: "White, Non-Hispanic",
    2: "Black, Non-Hispanic",
    4: "Hispanic/Latino",
    5: "Asian, Non-Hispanic",
    6: "Amer Ind/Alaska Native, Non-Hispanic",
    7: "Native Hawaiian/Other Pacific Islander, Non-Hispanic",
    9: "Multiracial, Non-Hispanic",
}

In [7]:
df['ethcat'] = df['ethcat'].replace(ethcat_mapping)
df

,replicate,ethcat,gender,waiting_time
0,0,"White, Non-Hispanic",M,47.499014
1,0,"White, Non-Hispanic",F,53.461349
2,0,"White, Non-Hispanic",M,58.934883
3,0,"White, Non-Hispanic",M,71.512356
4,0,Hispanic/Latino,F,74.343190
...,...,...,...,...
1148,29,"Black, Non-Hispanic",M,2581.607784
1149,29,"White, Non-Hispanic",M,2586.552961
1150,29,"White, Non-Hispanic",F,2587.226853
1151,29,"White, Non-Hispanic",M,2604.032224


In [8]:
(df.groupby(by=['replicate', 'ethcat']).agg(waiting_time=('waiting_time', 'mean'), count=('waiting_time', 'size'))
                                      .reset_index()
)#.to_csv('simulation_results.csv')

,replicate,ethcat,waiting_time,count
0,0,"Black, Non-Hispanic",422.300712,7
1,0,Hispanic/Latino,328.305134,4
2,0,"White, Non-Hispanic",352.359995,14
3,1,"Amer Ind/Alaska Native, Non-Hispanic",733.528766,1
4,1,"Asian, Non-Hispanic",382.526671,1
...,...,...,...,...
127,28,"White, Non-Hispanic",191.470550,9
128,29,"Asian, Non-Hispanic",2561.723279,1
129,29,"Black, Non-Hispanic",1102.084770,15
130,29,Hispanic/Latino,170.099188,8
